# Velocity graphs

Okay, so I've spent a lot of today working more on this idea of a structure for representing stress patterns in a flexible way.

I'll just jump to where I'm at so far, because I really like where I've brought this idea.

Basically, the idea is that you have something like automation curves - a set of points distributed over time that are interpolated between, and where a note's onset sits on this curve determines the velocity that it triggers with.

That's the first part of the idea, is just the mechanics of defining a set of points, how they're interpolated, and an algorithm that calculates the velocity value at any given point in time.

The second part of the idea is designing a procedure for generating a cyclical pattern of points.

I will start with the first part of the idea, because the second part of the idea is dependent on its existence.

## Part 1: Building the graph class

I want to have a class `VelocityGraph`. The primary function of the graph is this: you evaluate it at a specific time, and it returns a velocity value.

I see this consisting of a few components.

The first is the set of points. This will just be a list of ordered pairs, `(onset, velocity)`.

The second will be a set of interpolation algorithms. Which interpolation algorithm you use determines what velocity value is returned when evaluating the graph at a specific time.

### Sample and hold interpolation

This seems to be the simplest algorithm. You simply find the point p nearest to t that satisfies p[0] <= t, and return p[1].

How do we find the nearest point? Well, I guess we iterate through the points to start. 

With any of these algorithms, if there aren't any points, then we just return a value of 1 by default. 

Let's keep things simple by assuming the first point is at t=0, and there are points at every integer.

Let's say we are evaluating at t=5 1/2.

So we start at point 0. Is 0 >= 5 1/2? No? We move on.

...Finally, we get to point 6 >= 5 1/2? Yes? Okay! Return its velocity value.

Great. What if the first point isn't at t=0? We just do the same thing, don't we?

Say its at t=3 1/2, but we eval at t=2. What's the default behavior for evaluating before the first point when the first point isn't at t=0?

We just return the first point.

So, if t <= p[0], and p is the first point, then return p[1]. Otherwise, do the previous algorithm.

In [ ]:
from dataclasses import dataclass, field

from harmonica.utility._mixed import Mixed


@dataclass
class VelocityGraph:
    """Evaluates velocity values for notes based on their onset time.

    Consists of a set of points as ordered pairs `(onset, velocity)`.

    These points are interpolated between to retrieve velocity values.

    If there are no points, defaults to outputting 1."""

    points: list[tuple[Mixed, Mixed]] = field(default_factory=list)

    def __post_init__(self):
        self.points.sort(key=lambda x: x[0])

    def add_point(self, point: tuple[Mixed, Mixed]):
        self.points.append(point)
        self.points.sort(key=lambda x: x[0])

    def eval_sample_and_hold(self, t: Mixed) -> Mixed:
        if self.points == []:
            return Mixed(1)

        # If the time we're evaluating at occurs before the first point,
        # return the velocity value of the first point.
        if t < self.points[0][0]:
            return self.points[0][1]

        for i, point in enumerate(self.points):
            if point[0] > t:
                return self.points[i - 1][1]

        # If all else fails, return the velocity of the final point
        return self.points[-1][1]

Let's test this. I'll fill a graph with some random points. Importantly, there will be a point `(4 1/2, 2/3)`. I will evaluate `t=5`, and it should return `2/3`.

In [ ]:
from harmonica.utility._mixed import Mixed


graph = VelocityGraph(
    [
        (Mixed(1), Mixed("1/3")),
        (Mixed("1 1/2"), Mixed(1)),
        (Mixed("1 3/4"), Mixed(0)),
        (Mixed(2), Mixed(1)),
        (Mixed(3), Mixed("1/4")),
        (Mixed("4 1/2"), Mixed("2/3")),
        (Mixed(6), Mixed("4/5")),
    ]
)

graph.eval_sample_and_hold(Mixed(5))

Mixed('2/3')

Okay, seems to work great. 

Let's work on lerping--

WAIT! PAUSE!

Okay, I like the direction I'm going in here, for sure.

But like, isn't there a problem in this approach?

Okay, so, the original idea is to create this theoretically infinite cyclical pattern, right? 

So why would I build such a graph with a list of points? Doesn't that mean I have to _populate_ the list with objects in order to evaluate?

See what I mean? 

Okay, as an analogy... imagine TimeFuncs, except instead of entering an index and calculating the return value on the spot, you first had to populate a list full of every possible return value.

Isn't this kind of dumb? 

## Rethinking things

Well, let's think through this. If I carried on with this approach, how would it actually be used?

Obviously, at some point, you have to populate a container full of data. For example, time functions may calculate values in a lazy "on-demand" way, sure, but eventually, you DO have to populate a clip full of events, and those events have onset times, and those onset times are calculated by a time function.

So, let's think backwards here from our use case. We have a piano part or something, and we want to impose a velocity pattern over it. 

So we're going to iterate through each note in the clip, and for each note, we ask "what is the velocity at this time in our graph?"

If we took our current approach, I guess that would mean finding two points, p1 and p2, that surround our time t.

That is to say, we are looking for the two closest neighbors of t that satisfy p1 <= t <= p2. 

But the thing is, we have to calculate those points on the spot based on an algorithm.

Then, once we finally do calculate those (somehow), we have to interpolate between them. This step depends on the interpolation algorithm we're using.

One thing's for sure: I don't think it makes sense to structure this as a list of points sitting in memory, because this is something that will be calculated on the spot, for the most part.

## Skipping to part 2

Alright, I think we HAVE to think about the other part of this, before we can continue. 

How do we generate points on demand?

Let's list our parameters.

- Length of pattern
- Subdivision pattern
- Number of subdivision iterations
- Max and min velocities
- Velocity curve (how fast velocity diminishes with each subdivision)

So, you could have a simple pattern that's like this: [Strong, Weak].

The pattern is 1 beat long. The min and max velocities are 0 and 1. The velocity curve is linear.

Here's what no iterations would look like:
```
0, 1
```

1 iterations:
```
0,   1
1/2, 0
```

2 iterations:
```
0,   1
1/4, 0
1/2, 1/2
3/4, 0
```

3 iterations:
```
0,   1
1/8, 0
1/4, 1/4
3/8, 0
1/2, 1/2
5/8, 0
3/4, 1/4
7/8, 0
```

And so on.

Another idea for how the velocities diminish, maybe just have them follow a geometric series? 
1st subdivision level is (or some other initial value), and then each subsequence subdivision level gets a value of the previous subdivision level's velocity times 1/2, or 1/4, or 3/4, or 2/3, or whatever you want.

TBH I kinda like that better.

Here's what no iterations looks like:
```
0, 1
```

1 iteration:
```
0,   1
1/2, 1/2
```

2 iterations:
```
0,   1
1/4, 1/4
1/2, 1/2
3/4, 1/4
```

3 iterations:
```
0,   1
1/8, 1/8
1/4, 1/4
3/8, 1/8
1/2, 1/2
5/8, 1/8
3/4, 1/4
7/8, 1/8
```

So, what's the algorithm for this?

I guess you'd have to view these as multiple different layers, where you're constructing a new layer on each iteration. 

So the 3 iterations graph would actually look like:

```
0,   1
---
1/2, 1/2
---
1/4, 1/4
3/4, 1/4
---
1/8, 1/8
3/8, 1/8
5/8, 1/8
7/8, 1/8
```

So, how do you build these layers?

To answer that, let's look at a more complicated example with an irregular pattern, a 5/4 thing.

The length of the pattern is 5/4 instead of 1. 

Just lemme feel this out.

```
0, 1
```

```
0,   1
3/4, 1/2
```

```
0,   1
1/4, 1/4
1/2, 1/4
3/4, 1/2
1,   1/4
```

```
0,     1
1/8,   1/8
1/4,   1/4
3/8,   1/8
1/2,   1/4
5/8,   1/8
3/4,   1/2
7/8,   1/8
1,     1/4
1 1/8, 1/8
```

You could also have the initial pattern consist of more than one point.